# import libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet, ResNet50, VGG19, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

D1107 21:24:30.558598949    1886 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D1107 21:24:30.558620120    1886 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D1107 21:24:30.558623628    1886 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D1107 21:24:30.558626126    1886 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D1107 21:24:30.558628515    1886 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D1107 21:24:30.558630715    1886 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D1107 21:24:30.558637170    1886 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D1107 21:24:30.

# Setup TPU

In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()


Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/

# Setup Dataset

In [3]:
#Define the directory where your dataset is stored
dataset_directory = '/kaggle/input/plant-disease-expert/Image Data base/Image Data base'

# Get the list of class folders in the dataset directory
class_folders = [folder for folder in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, folder))]

# Initialize empty lists to store file paths for training and test sets
train_file_paths = []
test_file_paths = []

# Loop through each class folder and split the images into training and test sets
for class_folder in class_folders:
    class_directory = os.path.join(dataset_directory, class_folder)

    # Get the list of file paths for images in the current class folder
    class_file_paths = [os.path.join(class_directory, filename) for filename in os.listdir(class_directory) if filename.endswith('.jpg')]

    # Split the class data into training and test sets
    class_train_paths, class_test_paths = train_test_split(class_file_paths, test_size=0.05, random_state=42)

    # Add the split paths to the overall lists
    train_file_paths.extend(class_train_paths)
    test_file_paths.extend(class_test_paths)

# Define the directories for storing training and test images
train_directory = '/kaggle/working/train/'
test_directory = '/kaggle/working/test/'

# Move training images to the train directory
for train_path in train_file_paths:
    filename = os.path.basename(train_path)
    class_folder = os.path.basename(os.path.dirname(train_path))
    target_path = os.path.join(train_directory, class_folder, filename)
    os.makedirs(os.path.dirname(target_path), exist_ok=True)
    shutil.copy(train_path, target_path)

# Move test images to the test directory
for test_path in test_file_paths:
    filename = os.path.basename(test_path)
    class_folder = os.path.basename(os.path.dirname(test_path))
    target_path = os.path.join(test_directory, class_folder, filename)
    os.makedirs(os.path.dirname(target_path), exist_ok=True)
    shutil.copy(test_path, target_path)


#  Resize the dataset

In [4]:
target_size = (224, 224) # Change this to the desired size for your model input

def check_image_validity(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
    except (IOError, SyntaxError) as e:
        print(f"Invalid image found at {image_path}. Error: {e}")
        return False
    return True

# Loop through each class folder in the dataset
for class_folder in os.listdir(train_directory):
    class_folder_path = os.path.join(train_directory, class_folder)
    
    # Loop through each image in the class folder
    for filename in os.listdir(class_folder_path):
        if filename.endswith('.jpg'): # Assuming your images have the .jpg extension
            # Check if the image is valid
            image_path = os.path.join(class_folder_path, filename)
            if check_image_validity(image_path):
                # Open the image using PIL
                image = Image.open(image_path)
                im = image.convert('RGB')
                
                # Resize the image
                resized_image = im.resize(target_size, Image.Resampling.LANCZOS)
                
                # Save the resized image, overwrite the original image if needed
                resized_image.save(image_path)
                
for class_folder in os.listdir(test_directory):
    class_folder_path = os.path.join(train_directory, class_folder)
    
    # Loop through each image in the class folder
    for filename in os.listdir(class_folder_path):
        if filename.endswith('.jpg'): # Assuming your images have the .jpg extension
            # Check if the image is valid
            image_path = os.path.join(class_folder_path, filename)
            if check_image_validity(image_path):
                # Open the image using PIL
                image = Image.open(image_path)
                im = image.convert('RGB')
                
                # Resize the image
                resized_image = im.resize(target_size, Image.Resampling.LANCZOS)
                
                # Save the resized image, overwrite the original image if needed
                resized_image.save(image_path)


# Define Hyperparameters

In [5]:
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
NUM_CLASSES = 58
EPOCHS = 5
LR = 0.0009

# Data augmentation and preprocessing

In [6]:
train_directory = '/kaggle/working/train/'
test_directory = '/kaggle/working/test/'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.05
)

train_generator = train_datagen.flow_from_directory(
    directory=train_directory,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    directory=train_directory,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 180204 images belonging to 58 classes.
Found 9456 images belonging to 58 classes.


# Model Training

In [7]:

def create_mobilenet():
    with strategy.scope():
        base_model = MobileNet(weights = None, include_top=False, input_shape=(224, 224, 3))
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
        ])
        model.summary()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    

In [8]:
def create_vgg16net():
    with strategy.scope():
        base_model = VGG16(weights = None, include_top=False, input_shape=(224, 224, 3))
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
        ])
        model.summary()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    

In [9]:
def create_vgg19net():
    with strategy.scope():
        base_model = VGG19(weights = None, include_top=False, input_shape=(224, 224, 3))
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
        ])
        model.summary()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    

In [10]:
def create_resnet():
    with strategy.scope():
        base_model = ResNet50(weights = None, include_top=False, input_shape=(224, 224, 3))
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
        ])
        model.summary()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    

# Define the list of architectures to compare

In [11]:
architectures = ['MobileNet', 'ResNet50', 'VGG19', 'VGG16']
histories = []


# Loop through each architecture and train the model

In [ ]:
for architecture in architectures:
    print(f'Training {architecture}...')
    # Create a new model within the strategy scope
    if architecture == 'MobileNet':
        model = create_mobilenet()
    elif architecture == 'VGG16':
        model = create_vgg16net()
    elif architecture == 'VGG19':
        model = create_vgg19net()
    elif architecture == 'ResNet50':
        model = create_resnet()
    
    # Train the model using TPU
    history = model.fit(
        train_datagen.flow_from_directory(
            directory=train_directory,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            subset='training'
        ),
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        validation_data=train_datagen.flow_from_directory(
            directory=train_directory,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='categorical',
            subset='validation'
        ),
        validation_steps=val_generator.samples // BATCH_SIZE,
        epochs=EPOCHS
    )
    histories.append((architecture, history))
    # Save the trained model
    model.save(f'{architecture}_model.h5')

Training MobileNet...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 58)                59450     
                                                                 
Total params: 3,288,314
Trainable params: 3,266,426
Non-trainable params: 21,888
_________________________________________________________________
Found 180204 images belonging to 58 classes.
Found 9456 images belonging to 58 classes.
Epoch 1/5


2023-11-07 22:06:48.925364: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-11-07 22:06:49.322691: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


2815/2815 [==============================] - ETA: 0s - loss: 0.8664 - accuracy: 0.7454

2023-11-07 22:40:21.576678: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-11-07 22:40:21.756256: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


2815/2815 [==============================] - 2128s 748ms/step - loss: 0.8664 - accuracy: 0.7454 - val_loss: 1.2155 - val_accuracy: 0.6840
Epoch 2/5
2815/2815 [==============================] - 2096s 745ms/step - loss: 0.2733 - accuracy: 0.9126 - val_loss: 0.2854 - val_accuracy: 0.9221
Epoch 3/5
2815/2815 [==============================] - 2058s 731ms/step - loss: 0.1811 - accuracy: 0.9421 - val_loss: 0.2154 - val_accuracy: 0.9360
Epoch 4/5
2815/2815 [==============================] - 2074s 737ms/step - loss: 0.1361 - accuracy: 0.9554 - val_loss: 0.1759 - val_accuracy: 0.9456
Epoch 5/5
2815/2815 [==============================] - 2097s 745ms/step - loss: 0.1103 - accuracy: 0.9638 - val_loss: 0.3506 - val_accuracy: 0.8989
Training ResNet50...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                             

...

2815/2815 [==============================] - 2169s 750ms/step - loss: 0.7971 - accuracy: 0.7717 - val_loss: 1.9615 - val_accuracy: 0.7150
Epoch 2/5
2815/2815 [==============================] - 2085s 741ms/step - loss: 0.2581 - accuracy: 0.9181 - val_loss: 0.5619 - val_accuracy: 0.8406
Epoch 3/5
2815/2815 [==============================] - 2102s 747ms/step - loss: 0.1690 - accuracy: 0.9450 - val_loss: 0.4812 - val_accuracy: 0.8616
Epoch 4/5
2815/2815 [==============================] - 2079s 739ms/step - loss: 0.1249 - accuracy: 0.9593 - val_loss: 0.3903 - val_accuracy: 0.8866
Epoch 5/5
2815/2815 [==============================] - 2064s 733ms/step - loss: 0.0967 - accuracy: 0.9679 - val_loss: 0.1971 - val_accuracy: 0.9518
Training VGG19...
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                

2023-11-08 03:56:29.028761: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-11-08 03:56:29.242319: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


2815/2815 [==============================] - ETA: 0s - loss: 2.1691 - accuracy: 0.5391

2023-11-08 04:29:42.165861: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-11-08 04:29:42.302848: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


2815/2815 [==============================] - 2105s 741ms/step - loss: 2.1691 - accuracy: 0.5391 - val_loss: 0.8050 - val_accuracy: 0.7536
Epoch 2/5
1600/2815 [================>.............] - ETA: 14:26 - loss: 0.6353 - accuracy: 0.8013

# Plot the training and validation loss

In [ ]:
plt.figure(figsize=(12, 6))
for name, history in histories:
    plt.plot(history.history['loss'], label=f'{name} - Training Loss')
    plt.plot(history.history['val_loss'], label=f'{name} - Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss Comparison')
plt.show()

# Plot the training and validation accuracy

In [ ]:
plt.figure(figsize=(12, 6))
for name, history in histories:
    plt.plot(history.history['accuracy'], label=f'{name} - Training Accuracy')
    plt.plot(history.history['val_accuracy'], label=f'{name} - Validation Accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy Comparison')
plt.show()

# Test each model

## Load the testing data and preprocess it


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale without augmentation for testing data

test_generator = test_datagen.flow_from_list(
    test_file_paths,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

## Load the saved models

In [ ]:
models = []
for name in architectures:
    model = tf.keras.models.load_model(f'/kaggle/working/{name}_model.h5')
    models.append(model)

## Evaluate each model on the testing set

In [ ]:
for name, model in zip(architecture_names, models):
    print(f'Evaluating {name} model...')
    loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
    print(f'{name} Model - Test Accuracy: {accuracy * 100:.2f}%')